In [1]:
import os
import json
import numpy as np
import nibabel as nib
import imageio

In [2]:
dataset_dir = 'D:\\Dosyalar\\dataset\\Task06_Lung'
images_train_dir = os.path.join(dataset_dir, 'imagesTr')
labels_train_dir = os.path.join(dataset_dir, 'labelsTr')

In [3]:
output_dir = 'D:\\Dosyalar\dataset\\output_slices'
os.makedirs(output_dir, exist_ok=True)

In [ ]:
label_files = [f for f in os.listdir(labels_train_dir) if f.endswith('.nii.gz') and not f.startswith('._')]

In [ ]:
for label_file in label_files:
    label_path = os.path.join(labels_train_dir, label_file)
    label = nib.load(label_path).get_fdata()
    
    scan_number = label_file.split('_')[1].split('.')[0]
    
    scan_path = os.path.join(images_train_dir, label_file)
    if os.path.isfile(scan_path):
        ct_scan = nib.load(scan_path).get_fdata()

    tumor_slices_indexes = []

    # Check if both ct_scan and label are available
    if ct_scan is not None and label is not None:
        # Check each slice for tumor
        for slice_idx in range(label.shape[2]):
            if np.any(label[:, :, slice_idx]):  # Check if the label slice contains the tumor
                save_path = os.path.join(output_dir, f'scan_{scan_number}', 'label')
                os.makedirs(save_path, exist_ok=True)
                nib.save(nib.Nifti1Image(label[:, :, slice_idx], np.eye(4)),
                         os.path.join(save_path, f'slice_{slice_idx}.nii.gz'))
                imageio.imsave(os.path.join(save_path, f'slice_{slice_idx}.png'), label[:, :, slice_idx].astype(np.uint8)*255)

                save_path = os.path.join(output_dir, f'scan_{scan_number}', 'train')
                os.makedirs(save_path, exist_ok=True)
                nib.save(nib.Nifti1Image(ct_scan[:, :, slice_idx], np.eye(4)),
                         os.path.join(save_path, f'slice_{slice_idx}.nii.gz'))
                imageio.imsave(os.path.join(save_path, f'slice_{slice_idx}.png'), ct_scan[:, :, slice_idx].astype(np.uint8))

                # Add slice index to tumor_slices_indexes
                tumor_slices_indexes.append(slice_idx)

        parent_dir = os.path.join(output_dir, f'scan_{scan_number}')
        json_path = os.path.join(parent_dir, 'slice_indexes.json')
        with open(json_path, 'w') as json_file:
            json.dump(tumor_slices_indexes, json_file)